In [12]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from scipy import signal
import time
import datetime
import serial
from scipy.fftpack import fft,ifft
from matplotlib.pylab import mpl
import struct
import os
import serial
import binascii
from fun_chy2 import *

mpl.rcParams['font.sans-serif'] = ['SimHei']
mpl.rcParams['axes.unicode_minus'] = False

In [23]:
!python.exe -m PyQt5.uic.pyuic Automated_testingV1.3.ui -o Automated_testingV13.py

In [33]:
!python.exe -m PyQt5.uic.pyuic sample_testingV1.32.ui -o sample_testingV13.py

In [27]:
!python.exe -m PyQt5.uic.pyuic sample_testingV1.3.ui -o sample_testingV13.py

In [3]:
# xxxwrj 无人机惯导
def re_strings(data):
    split_num = 2
    crc_data = data
    lists = [crc_data[i*split_num:i*split_num+split_num].upper() for i in range(len(crc_data)//split_num)]
    return ''.join(lists[::-1])
# print(re_strings('0123456789'))

strings = ''
strings+='90EB'
strings+='1600' # 总字节-5

# strings+='0080' #北斗/GNSS 北斗
# strings+='0040' #北斗/GNSS GNSS
# strings+='0020'     # 混合
strings+='00A0' #北斗/GNSS 组合导航

# strings+=re_strings('8000')     # 指令字-对准
strings+=re_strings('8000')     # 指令字-对准

loc1 = 116.50270
loc2 = 39.73155
loc3 = 20
# loc1 = 0
# loc2 = 0
# loc3 = 0
strings+=re_strings(hex(int(loc1/90*1073741824))[2:].rjust(8,'0'))
strings+=re_strings(hex(int(loc2/90*1073741824))[2:].rjust(8,'0'))
strings+=re_strings(hex(int(loc3/16384*1073741824))[2:].rjust(8,'0'))
for i in range(3):
    strings+='0000'


split_num = 2
crc_data = strings
lists = [crc_data[i*split_num:i*split_num+split_num].upper() for i in range(len(crc_data)//split_num)]
int_lists = [int(num,16) for num in lists]
# print(len(int_lists[2:]))
checks = hex(sum(int_lists[2:]))[2:].rjust(2,'0')[-2:]
lists.append(checks.upper())
counts = []
for i in range(27):
    counts.append(str(i).rjust(2,'0'))
print(' '.join(counts))
print(' '.join(lists))
print('总长度:{}'.format(len(lists)))

00 01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26
90 EB 16 00 00 A0 00 80 57 AB D8 52 6F E8 40 1C 00 00 14 00 00 00 00 00 00 00 29
总长度:27


In [10]:
90 EB 16 00 00 A0 00 80 57 AB D8 52 6F E8 40 1C 00 00 14 00 00 00 00 00 00 00 29

# 组合导航 BD+GNSS
90 EB 16 00 00 E0 00 80 57 AB D8 52 6F E8 40 1C 00 00 14 00 00 00 00 00 00 00 69
# 组合导航 切换GNSS
90 EB 16 00 00 60 00 80 57 AB D8 52 6F E8 40 1C 00 00 14 00 00 00 00 00 00 00 E9
# 组合导航 切换BD
90 EB 16 00 00 A0 00 80 57 AB D8 52 6F E8 40 1C 00 00 14 00 00 00 00 00 00 00 29

# 纯惯 切换BD
90 EB 16 00 00 80 00 80 57 AB D8 52 6F E8 40 1C 00 00 14 00 00 00 00 00 00 00 09
# 纯惯 切换GNSS
90 EB 16 00 00 40 00 80 57 AB D8 52 6F E8 40 1C 00 00 14 00 00 00 00 00 00 00 C9


'0x509'

In [6]:
# CRC32 预计算表
def calculate_crc32(buffer: bytearray) -> int:
    """
    计算给定二进制缓冲区的 CRC32 值
    :param buffer: 二进制缓冲区（bytearray）
    :return: 计算得到的 CRC32 值
    """
    ul_crc = 0
    for byte in buffer:
        ul_crc = aul_crc_table[(ul_crc ^ byte) & 0xff] ^ (ul_crc >> 8)
    return ul_crc



In [28]:
lists_data = '''
 0x00000000UL, 0x77073096UL, 0xee0e612cUL, 0x990951baUL, 0x076dc419UL,
0x706af48fUL,
 0xe963a535UL, 0x9e6495a3UL, 0x0edb8832UL, 0x79dcb8a4UL, 0xe0d5e91eUL,
0x97d2d988UL,
 0x09b64c2bUL, 0x7eb17cbdUL, 0xe7b82d07UL, 0x90bf1d91UL, 0x1db71064UL,
0x6ab020f2UL,
 0xf3b97148UL, 0x84be41deUL, 0x1adad47dUL, 0x6ddde4ebUL, 0xf4d4b551UL,
0x83d385c7UL,
 0x136c9856UL, 0x646ba8c0UL, 0xfd62f97aUL, 0x8a65c9ecUL, 0x14015c4fUL,
0x63066cd9UL,
 0xfa0f3d63UL, 0x8d080df5UL, 0x3b6e20c8UL, 0x4c69105eUL, 0xd56041e4UL,
0xa2677172UL,
 0x3c03e4d1UL, 0x4b04d447UL, 0xd20d85fdUL, 0xa50ab56bUL, 0x35b5a8faUL,
0x42b2986cUL,
 0xdbbbc9d6UL, 0xacbcf940UL, 0x32d86ce3UL, 0x45df5c75UL, 0xdcd60dcfUL,
0xabd13d59UL,
 0x26d930acUL, 0x51de003aUL, 0xc8d75180UL, 0xbfd06116UL, 0x21b4f4b5UL,
0x56b3c423UL,
 0xcfba9599UL, 0xb8bda50fUL, 0x2802b89eUL, 0x5f058808UL, 0xc60cd9b2UL,
0xb10be924UL,
 0x2f6f7c87UL, 0x58684c11UL, 0xc1611dabUL, 0xb6662d3dUL, 0x76dc4190UL,
0x01db7106UL,
 0x98d220bcUL, 0xefd5102aUL, 0x71b18589UL, 0x06b6b51fUL, 0x9fbfe4a5UL,
0xe8b8d433UL,
 0x7807c9a2UL, 0x0f00f934UL, 0x9609a88eUL, 0xe10e9818UL, 0x7f6a0dbbUL,
0x086d3d2dUL,
 0x91646c97UL, 0xe6635c01UL, 0x6b6b51f4UL, 0x1c6c6162UL, 0x856530d8UL,
0xf262004eUL,
 0x6c0695edUL, 0x1b01a57bUL, 0x8208f4c1UL, 0xf50fc457UL, 0x65b0d9c6UL,
0x12b7e950UL,
 0x8bbeb8eaUL, 0xfcb9887cUL, 0x62dd1ddfUL, 0x15da2d49UL, 0x8cd37cf3UL,
0xfbd44c65UL,
 0x4db26158UL, 0x3ab551ceUL, 0xa3bc0074UL, 0xd4bb30e2UL, 0x4adfa541UL,
0x3dd895d7UL,
 0xa4d1c46dUL, 0xd3d6f4fbUL, 0x4369e96aUL, 0x346ed9fcUL, 0xad678846UL,
0xda60b8d0UL,
 0x44042d73UL, 0x33031de5UL, 0xaa0a4c5fUL, 0xdd0d7cc9UL, 0x5005713cUL,
0x270241aaUL,
 0xbe0b1010UL, 0xc90c2086UL, 0x5768b525UL, 0x206f85b3UL, 0xb966d409UL,
0xce61e49fUL,
 0x5edef90eUL, 0x29d9c998UL, 0xb0d09822UL, 0xc7d7a8b4UL, 0x59b33d17UL,
0x2eb40d81UL,
 0xb7bd5c3bUL, 0xc0ba6cadUL, 0xedb88320UL, 0x9abfb3b6UL, 0x03b6e20cUL,
0x74b1d29aUL,
 0xead54739UL, 0x9dd277afUL, 0x04db2615UL, 0x73dc1683UL, 0xe3630b12UL,
0x94643b84UL,
 0x0d6d6a3eUL, 0x7a6a5aa8UL, 0xe40ecf0bUL, 0x9309ff9dUL, 0x0a00ae27UL,
0x7d079eb1UL,
 0xf00f9344UL, 0x8708a3d2UL, 0x1e01f268UL, 0x6906c2feUL, 0xf762575dUL,
0x806567cbUL,
 0x196c3671UL, 0x6e6b06e7UL, 0xfed41b76UL, 0x89d32be0UL, 0x10da7a5aUL,
0x67dd4accUL,
 0xf9b9df6fUL, 0x8ebeeff9UL, 0x17b7be43UL, 0x60b08ed5UL, 0xd6d6a3e8UL,
0xa1d1937eUL,
 0x38d8c2c4UL, 0x4fdff252UL, 0xd1bb67f1UL, 0xa6bc5767UL, 0x3fb506ddUL,
0x48b2364bUL,
 0xd80d2bdaUL, 0xaf0a1b4cUL, 0x36034af6UL, 0x41047a60UL, 0xdf60efc3UL,
0xa867df55UL,
 0x316e8eefUL, 0x4669be79UL, 0xcb61b38cUL, 0xbc66831aUL, 0x256fd2a0UL,
0x5268e236UL,
 0xcc0c7795UL, 0xbb0b4703UL, 0x220216b9UL, 0x5505262fUL, 0xc5ba3bbeUL,
0xb2bd0b28UL,   
 0x2bb45a92UL, 0x5cb36a04UL, 0xc2d7ffa7UL, 0xb5d0cf31UL, 0x2cd99e8bUL,
0x5bdeae1dUL,
 0x9b64c2b0UL, 0xec63f226UL, 0x756aa39cUL, 0x026d930aUL, 0x9c0906a9UL,
0xeb0e363fUL,
 0x72076785UL, 0x05005713UL, 0x95bf4a82UL, 0xe2b87a14UL, 0x7bb12baeUL,
0x0cb61b38UL,
 0x92d28e9bUL, 0xe5d5be0dUL, 0x7cdcefb7UL, 0x0bdbdf21UL, 0x86d3d2d4UL,
0xf1d4e242UL,
 0x68ddb3f8UL, 0x1fda836eUL, 0x81be16cdUL, 0xf6b9265bUL, 0x6fb077e1UL,
0x18b74777UL,
 0x88085ae6UL, 0xff0f6a70UL, 0x66063bcaUL, 0x11010b5cUL, 0x8f659effUL,
0xf862ae69UL,
 0x616bffd3UL, 0x166ccf45UL, 0xa00ae278UL, 0xd70dd2eeUL, 0x4e048354UL,
0x3903b3c2UL,
 0xa7672661UL, 0xd06016f7UL, 0x4969474dUL, 0x3e6e77dbUL, 0xaed16a4aUL,
0xd9d65adcUL,
 0x40df0b66UL, 0x37d83bf0UL, 0xa9bcae53UL, 0xdebb9ec5UL, 0x47b2cf7fUL,
0x30b5ffe9UL,
 0xbdbdf21cUL, 0xcabac28aUL, 0x53b39330UL, 0x24b4a3a6UL, 0xbad03605UL,
0xcdd70693UL,
 0x54de5729UL, 0x23d967bfUL, 0xb3667a2eUL, 0xc4614ab8UL, 0x5d681b02UL,
0x2a6f2b94UL,
 0xb40bbe37UL, 0xc30c8ea1UL, 0x5a05df1bUL, 0x2d02ef8dUL
'''
lists_data = lists_data.replace('UL','')
lists_data = lists_data.replace('\n','')
lists_data = lists_data.replace(' ','')
lists_data = lists_data.split(',')
lists_data = [int(i,16) for i in lists_data]
print(lists_data)

[0, 1996959894, 3993919788, 2567524794, 124634137, 1886057615, 3915621685, 2657392035, 249268274, 2044508324, 3772115230, 2547177864, 162941995, 2125561021, 3887607047, 2428444049, 498536548, 1789927666, 4089016648, 2227061214, 450548861, 1843258603, 4107580753, 2211677639, 325883990, 1684777152, 4251122042, 2321926636, 335633487, 1661365465, 4195302755, 2366115317, 997073096, 1281953886, 3579855332, 2724688242, 1006888145, 1258607687, 3524101629, 2768942443, 901097722, 1119000684, 3686517206, 2898065728, 853044451, 1172266101, 3705015759, 2882616665, 651767980, 1373503546, 3369554304, 3218104598, 565507253, 1454621731, 3485111705, 3099436303, 671266974, 1594198024, 3322730930, 2970347812, 795835527, 1483230225, 3244367275, 3060149565, 1994146192, 31158534, 2563907772, 4023717930, 1907459465, 112637215, 2680153253, 3904427059, 2013776290, 251722036, 2517215374, 3775830040, 2137656763, 141376813, 2439277719, 3865271297, 1802195444, 476864866, 2238001368, 4066508878, 1812370925, 45309273

In [34]:
len(lists_data)/8

32.0

In [37]:
for j in range(32):
    for i in range(8):
        print(str(lists_data[j*8+i])+',',end='' )
    print()
    

0,1996959894,3993919788,2567524794,124634137,1886057615,3915621685,2657392035,
249268274,2044508324,3772115230,2547177864,162941995,2125561021,3887607047,2428444049,
498536548,1789927666,4089016648,2227061214,450548861,1843258603,4107580753,2211677639,
325883990,1684777152,4251122042,2321926636,335633487,1661365465,4195302755,2366115317,
997073096,1281953886,3579855332,2724688242,1006888145,1258607687,3524101629,2768942443,
901097722,1119000684,3686517206,2898065728,853044451,1172266101,3705015759,2882616665,
651767980,1373503546,3369554304,3218104598,565507253,1454621731,3485111705,3099436303,
671266974,1594198024,3322730930,2970347812,795835527,1483230225,3244367275,3060149565,
1994146192,31158534,2563907772,4023717930,1907459465,112637215,2680153253,3904427059,
2013776290,251722036,2517215374,3775830040,2137656763,141376813,2439277719,3865271297,
1802195444,476864866,2238001368,4066508878,1812370925,453092731,2181625025,4111451223,
1706088902,314042704,2344532202,4240017532,16586582

In [15]:
text = '''#DRPVAA,COM1,0,92.0,FINE,1867,111471.800,00000000,14,0;SOL_COMPUTED,SINGLE
,WGS84,0,0,0,0,0.800,2573.000,40.07899836523,116.23661722090,68.5569,-
9.7848,0.1522,0.1489,0.0213,-0.0001,-0.0001,0.0010,0.0096,0.0097,0.0097,359.589710,-
1.236787,1.344697,0.0000,0.0020,0.0020,1,0,0,0,0.0000,0.0000,0.0000,0.0000,0.000000,0.
000000,0.000000,0.000000*9aa8b508'''
text = text.replace('\n','')
data = text[1:].split('*')[0]
crc = text[1:].split('*')[1]
print(data)
print(len(data.split(',')))
print(crc)
crc = binascii.crc32(data.encode('ascii'))
print(crc)
print(hex(crc))
crc2 = calculate_crc32(data.encode('ascii'))
print(crc2)
print(hex(crc2))

DRPVAA,COM1,0,92.0,FINE,1867,111471.800,00000000,14,0;SOL_COMPUTED,SINGLE,WGS84,0,0,0,0,0.800,2573.000,40.07899836523,116.23661722090,68.5569,-9.7848,0.1522,0.1489,0.0213,-0.0001,-0.0001,0.0010,0.0096,0.0097,0.0097,359.589710,-1.236787,1.344697,0.0000,0.0020,0.0020,1,0,0,0,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000
49
9aa8b508
3482309119
0xcf8fd1ff
2594747656
0x9aa8b508


In [24]:
text = 'HEADINGA,COM1,0,0.0,0,0,0.000,0,0,0;,INSUFFICIENT_OBS,NONE,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,"0",0,0,0,0,0,00,0,0'
print(len(text.split(',')))
crc2 = calculate_crc32(text.encode('ascii'))
print(crc2)
print(hex(crc2))

27
1952116701
0x745aefdd


In [22]:
text.split(',')[25]

'0'